# 1. Imports

In [2]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

# 2. Importing the Datasets

In [5]:
ddos_portmap_2018_df = pd.read_csv("../data/train_test/CSE_CIC_IDS2018/ddos_portmap_2018_train.csv")

In [6]:
ddos_ldap_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_ldap_2019_train.csv")

In [7]:
ddos_netbios_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_netbios_2019_train.csv")

In [8]:
ddos_syn_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_syn_2019_train.csv")

In [9]:
ddos_udp_2019_df = pd.read_csv("../data/train_test/CIC_DDoS2019/ddos_udp_2019_train.csv")

In [10]:
target_feature = 'Label'

# 3. Feature Selection Method

In [1]:
def stableFeatureSelect(data_df, num_splits):
    
    df_split = np.array_split(data_df, num_splits)
    
    for small_df in df_split:
        # perform feature selection and return ranking
        
    # add ranks together and return rankings and save
    # each feature of the original dataset will have ranking 
    # ex. Feature_A 0.2
    # ex. Feature_B 0.4
    # ex. Feature_C 0.0

SyntaxError: unexpected EOF while parsing (<ipython-input-1-7fa713618edd>, line 10)

In [ ]:
portmap_ranking = stableFeatureSelect(ddos_portmap_2018_df, 5)

In [ ]:
# Save portmap_ranking in csv

# 4. Saving the Ranking

CODE TO SAVE A CSV FILE WITH:
- 'feature' is a column of all of the features in the original dataset
- 'ranking' is a column with the rankings (in range 0 to 1) of the feature